# Intro
Welcome to the monthly Kaggle experiment in 2021. This is [march](https://www.kaggle.com/c/tabular-playground-series-mar-2021/overview). 
![](https://storage.googleapis.com/kaggle-competitions/kaggle/25225/logos/header.png)

This notebook is a simple tutorial of the second experimental competition. For feature encoding techniques we recommend [this notebook](https://www.kaggle.com/drcapa/categorical-feature-engineering-2-xgb).

<span style="color: royalblue;">Please vote the notebook up if it helps you. Feel free to leave a comment above the notebook. Thank you. </span>

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import eli5
from eli5.sklearn import PermutationImportance

import warnings
warnings.filterwarnings("ignore")

# Path

In [ ]:
path = '/kaggle/input/tabular-playground-series-mar-2021/'
os.listdir(path)

# Load Data

In [ ]:
train_data = pd.read_csv(path+'train.csv')
test_data = pd.read_csv(path+'test.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

# Overview

In [ ]:
print('Number train samples:', len(train_data.index))
print('Number test samples:', len(test_data.index))
print('Number features:', len(train_data.columns))

In [ ]:
print('Missing values on the train data:', train_data.isnull().sum().sum())
print('Missing values on the test data:', test_data.isnull().sum().sum())

# Feature Engineering

In [ ]:
features_cat = ['cat'+str(i) for i in range(19)]
features_num = ['cont'+str(i) for i in range(11)]
no_features = ['id', 'target']

In [ ]:
print('number of categorical features:', len(features_cat))
print('number of numerical features:', len(features_num))

Encoding of categorical features:

In [ ]:
col = features_cat[10]

In [ ]:
le = LabelEncoder()
for col in features_cat:
    le.fit(train_data[col].append(test_data[col]))
    train_data[col] = le.transform(train_data[col])
    test_data[col] = le.transform(test_data[col])

We create statistical features like mean, max and min for every sample on the train and test data.

In [ ]:
train_data['mean'] = train_data[features_num].mean(axis=1)
train_data['std'] = train_data[features_num].std(axis=1)
train_data['max'] = train_data[features_num].max(axis=1)
train_data['min'] = train_data[features_num].min(axis=1)
train_data['sum'] = train_data[features_num].sum(axis=1)

test_data['mean'] = test_data[features_num].mean(axis=1)
test_data['std'] = test_data[features_num].std(axis=1)
test_data['max'] = test_data[features_num].max(axis=1)
test_data['min'] = test_data[features_num].min(axis=1)
test_data['sum'] = test_data[features_num].sum(axis=1)

# EDA

Distribution of the numerical values:

In [ ]:
train_data.boxplot(column=features_num, figsize=(10,4))
plt.show()

Distribution of the categorcial data:

In [ ]:
train_data.boxplot(column=features_cat, figsize=(10,4))
plt.show()

Correlation matrix:

In [ ]:
temp = train_data
corr = temp.corr()
#corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

# Train, Validation And Test Data

In [ ]:
X_train = train_data[train_data.columns.difference(no_features)]
y_train = train_data['target']
X_test = test_data[test_data.columns.difference(no_features)]

Scale Date:

In [ ]:
mean = X_train.mean()
X_train = X_train-mean
std = X_train.std()
X_train = X_train/std
X_test = (X_test-mean)/std

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=2021)

In [ ]:
print('Train shape:', X_train.shape)
print('Val shape:', X_val.shape)
print('Test shape:', X_test.shape)

# Model

In [ ]:
model = RandomForestClassifier(n_estimators=300,
                               max_depth=7,
                               random_state=2021)

In [ ]:
model.fit(X_train, y_train)
y_val_pred = model.predict_proba(X_val)
print('Score validation data:', roc_auc_score(y_val, y_val_pred[:, 1]))

# Analyse Training

Feature Importance:

In [ ]:
importance = model.feature_importances_
fig = plt.figure(figsize=(10, 6))
x = list(train_data[train_data.columns.difference(no_features)])
plt.barh(x, 100*importance, color='orange')
plt.title('Feature Importance', loc='left')
plt.xlabel('Percentage')
plt.grid()
plt.show()

Permutation importance:

In [ ]:
perm = PermutationImportance(model, random_state=2021).fit(X_val, y_val)
eli5.show_weights(perm, feature_names = X_val.columns.tolist())

# Predict Test Data

In [ ]:
y_test = model.predict_proba(X_test)

In [ ]:
output = samp_subm.copy()
output['target'] = y_test[:, 1]

# Write Output

In [ ]:
output.to_csv('submission.csv', index=False)

In [ ]:
output.head()